a. La fuerza de Lorentz es:
$$\vec{F}=q(\vec{E}+\vec{v}\times\vec{B})$$
Se sabe que no hay campo eléctrico, y el campo magnético es:
$$\vec{B}=B_0 sin(\omega t)\hat{k}$$
Asumiendo que la velocidad es:
$$\vec{v}=v_x\hat{i}+v_y\hat{j}+v_z\hat{k}$$
Entonces la fuerza sería:
$$\vec{F}=B_0 q v_y sin(\omega t)\hat{i}-B_0 q v_x sin(\omega t)\hat{j}$$
Y las componentes de la aceleración serían:
$$a_x=-k v_y sin(\omega t)$$
$$a_y=k v_x sin(\omega t)$$
Donde k es una constante que depende de la masa, la carga y el campo magnético inicial.

In [2]:
import numpy
import matplotlib.pyplot as plt
k=1
w=0.3
def dvx(r,t):
    return -k*r[4]*numpy.sin(w*t)
def dvy(r,t):
    return k*r[3]*numpy.sin(w*t)
def dvz(r,t):
    return 0
def dx(r,t):
    return r[3]
def dy(r,t):
    return r[4]
def dz(r,t):
    return r[5]
def dr(r,t):
    return numpy.array(dx(r,t),dy(r,t),dz(r,t),dvx(r,t),dvy(r,t),dvz(r,t))

In [ ]:
def rungekutta(h,t,r0):
    t=numpy.arange(0,t,h)
    R=numpy.zeros((t.size,6))
    R[0]=r0
    for i in range(0,t.size-1):
        k1=dr(R[i],t[i])
        k2=dr(R[i]+k1*h,t[i]+h)
        R[i+1]=R[i]((k1+k2)*h/2)